# Bulk Loading

Previously, we inserted sixty records into the readings table but often we have much bigger sets of data.

The 'flight-data.csv' contains data for over 200 000 readings. Let's learn how to load that into our database.

First we connect to our database:

In [30]:
import sqlalchemy as sa
import pandas as pd
engine = sa.create_engine('sqlite:///flight.db')
connection = engine.connect()
connection.execute("PRAGMA foreign_keys=on")

And then we load the contents of the file into a pandas dataframe:

In [45]:
flight_data = pd.read_csv('flight-data.csv')

Let's try using the same technique as when we loaded the sixty records in Chapter 3.

We defined an INSERT statement using bound parameters and then looped over our dataset and executed that statement once for each record.

We'll do that again now but with the loop inside a function that we call so that we can time it.

**NOTE** If you are using the online service, it is best not to run this example as it will take a significant length of time.

**NOTE** There are some errors in the humidity readings due to sensor glitches. We have to use Python's min and max functions to ensure we don't breach the constraints we created on our table.

In [5]:
sql = """
    INSERT INTO readings
        (flight, ts, temp, pressure, humidity,
        accel_x, accel_y, accel_z)
    VALUES
        ('hab1', ?, ?, ?, ?, ?, ?, ?)
"""

def load_data(connection, data):

    for row in data.itertuples():
        connection.execute(sql, (
            row.timestamp, row.temp_h, row.pressure,
            min(100, max(0, row.humidity)),
            row.accel_x, row.accel_y, row.accel_z
        ))

connection.execute("DELETE FROM readings")     
%timeit -n1 -r1 load_data(connection, flight_data)

5min 38s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


To understand why this technique takes so long to run, we need to understand database transactions.

When you ask the DBMS to make a change to a table using an INSERT, UPDATE or DELETE statement, it will guarantee that:

* Atomicity - The entire change will either succeed or fail. It will never partially complete.
* Consistency - Your data will always be consistent. Your table constraints will never be violated.
* Isolation - Nobody else will see any of your changes until they have all completed.
* Durability - Once your changes are complete, they have been entirely written to the underlying storage.

These guarantees are known as ACID transactions.

You can make as many changes as you want within a single transaction. If, for example, it is important that records inserted into two tables must either succeed or fail together (e.g. an invoice header and invoice detail table in an accounting system), then it would be sensible to make those inserts within a single transaction.

Unless told otherwise, SQLite creates a transaction for each and every INSERT, UPDATE and DELETE statement and, because it has to guarantee the durability, it has to write to to the database file each. This repeated disk writing is the reason our first example is slow.

Instead, let's implicitly specify that all our inserts should take place in a single transaction, and therefore only one write to disk.

We specify a transaction using our connection object's `begin` method:

In [7]:
def load_data(connection, data):

    with connection.begin():
        for row in data.itertuples():
            connection.execute(sql, (
                row.timestamp, row.temp_h, row.pressure,
                min(100, max(0, row.humidity)),
                row.accel_x, row.accel_y, row.accel_z
            ))
            
connection.execute("DELETE FROM readings")
%timeit -n1 -r1 load_data(connection, flight_data)

11.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


You should see a huge drop in the amount of time it takes to execute this second example.

But we can do better. At the moment, we are using a for loop to iterate over the pandas dataframe. Instead, let's use a list comprehension:

In [9]:
def load_data(connection, data):

    data = [
        (row.timestamp, row.temp_h, row.pressure,
         min(100, max(0, row.humidity)),
         row.accel_x, row.accel_y, row.accel_z)
        for row in data.itertuples()
    ]
    with connection.begin():
        connection.execute(sql, data)
        
connection.execute("DELETE FROM readings")     
%timeit -n1 -r1 load_data(connection, flight_data)

3.02 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


On my laptop, that's now 100x faster than our first example and 4x faster than the second.

Finally, although this tutorial focuses on learning SQL, it is worth noting that a pandas dataframe can be loaded directly into a database table using the `to_sql` method.

We need to tidy up the dataframe first, but let's see how that performs:

In [46]:
flight_data.drop(['temp_p', 'pitch', 'roll', 'yaw', 'mag_x', 'mag_y', 'mag_z', 'gyro_x', 'gyro_y', 'gyro_z'], axis=1, inplace=True)
flight_data['humidity'] = flight_data['humidity'].clip(0, 100)
flight_data = flight_data.rename(columns={'temp_h': 'temp', 'timestamp': 'ts'})
flight_data['flight'] = 'hab1'

connection.execute("DELETE FROM readings")
%timeit -n1 -r1 flight_data.to_sql('readings', connection, if_exists='append', index=False)

5.53 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
